<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
%matplotlib inline
import numpy
import matplotlib.pyplot as plt

# Boundary Value Problems:  Discretization

## Model Problems

The simplest boundary value problem (BVP) we will run into is the one-dimensional version of Poisson's equation
$$
    u''(x) = f(x).
$$

Usually we solve this equation on a finite interval with either Dirichlet or Neumann boundary condtions.  Because there are two derivatives in the equation we need two boundary conditions to solve the PDE (really and ODE in this case) uniquely.  To start let us consider the following basic problem
$$\begin{aligned}
    u''(x) = f(x) ~~~ \Omega = [a, b] \\
    u(a) = \alpha ~~~ u(b) = \beta.
\end{aligned}$$

BVPs of this sort are often the result of looking at the steady-state form of a time dependent PDE.  For instance, if we were considering the steady-state solution to the heat equation
$$
    u_t(x,t) = \kappa u_xx(x,t) + \Psi(x,t) ~~~~ \Omega = [0, T] \times [a, b] \\
    u(x, 0) = u^0(x) ~~~ u(a, t) = \alpha(t) ~~~ u(b, t) = \beta(t)
$$
we would solve the equation where $u_t = 0$ and arrive at
$$
    u''(x) = - \Psi / \kappa,
$$
a version of Poisson's equation above.

In higher spatial dimensions the second derivative turns into a Laplacian.  Notation varies for this but all these are equivalent statements:
$$\begin{aligned}
    \nabla^2 u(\vec{x}) &= f(\vec{x}) \\
    \Delta u(\vec{x}) &= f(\vec{x}) \\
    \sum^N_{i=1} u_{x_i x_i} &= f(\vec{x}).
\end{aligned}$$

## One-Dimensional Discretization

As a first approach to solving the one-dimensional Poisson's equation let's break up the domain into `m` points, often called a *mesh* or *grid*.  Our goal is to approximate the unknown function $u(x)$ as the mesh points $x_i$.  First we can relate the number of mesh points `m` to the distance between with
$$
    \Delta x = \frac{1}{m + 1}.
$$
The mesh points $x_i$ can be written as
$$
    x_i = a + i \Delta x.
$$

We can let $\Delta x$ vary and many of the formulas above have only minor modifications but we will leave that for homework.  Notationally we will also adopt the notation
$$
    U_i \approx u(x_i)
$$
so that $U_i$ are the approximate solution at the grid points and retain the lower-case $u$ to denote the true solution.

To simplify our discussion let's consider the ODE
$$
    u''(x) = f(x) ~~~ \Omega = [0, 1] \\
    u(0) = \alpha ~~~ u(1) = \beta.
$$

Applying the 2nd order, centered difference approixmation for the 2d derivative we have the equation
$$
    D^2 U_i = \frac{1}{\Delta x^2} (U_{i+1} - 2 U_i + U_{i-1})
$$
so that we end up with the approximate algebraic expression at every grid point of
$$
    \frac{1}{\Delta x^2} (U_{i+1} - 2 U_i + U_{i-1}) = f(x_i)  ~~~ i = 1, 2, 3, \ldots, m.
$$

Note at this point that these algebraic equations are coupled as each $U_i$ depends on its neighbors.  This means we can write these as system of coupled equations
$$
    A U = F.
$$

#### Write the system of equations
$$
    \frac{1}{\Delta x^2} (U_{i+1} - 2 U_i + U_{i-1}) = f(x_i)  ~~~ i = 1, 2, 3, \ldots, m.
$$

Note the boundary conditions!

$$
    \frac{1}{\Delta x^2} \begin{bmatrix}
    -2 &  1 &    &    &    \\
     1 & -2 &  1 &    &    \\
       &  1 & -2 &  1 &    \\
       &    &  1 & -2 &  1 \\
       &    &    &  1 & -2 \\
    \end{bmatrix} \begin{bmatrix}
        U_1 \\ U_2 \\ U_3 \\ U_4 \\ U_5
    \end{bmatrix} = 
    \begin{bmatrix}
        f(x_1) - \frac{\alpha}{\Delta x^2} \\ f(x_2) \\ f(x_3) \\ f(x_4) \\ f(x_5) - \frac{\beta}{\Delta x^2} \\
    \end{bmatrix}.
$$

## Error Analysis

A natural question to ask given our approximation $U_i$ is how close this is to the true solution $u(x)$ at the grid points $x_i$.  To address this we will define the error $E$ as
$$
    E = U - \hat{U}
$$
where $U$ is the vector of the approximate solution and $\hat{U}$ is the vector composed of the $u(x_i)$.  This leaves $E$ as a vector still so often we ask the question how does the norm of $E$ behave given a particular $\Delta x$.  For the $\infty$-norm we would have
$$
    ||E||_\infty = \max_{1 \leq i \leq m} |E_i| = \max_{1 \leq i \leq m} |U_i - u(x_i)|
$$
If we can show that $||E||_\infty$ goes to zero as $\Delta x \rightarrow 0$ we can then claim that the approximate solution $U_i$ at any of the grid points $E_i \rightarrow 0$.  If we would like to use other norms we often define slightly modified versions of the norms that also contain the grid width $\Delta x$ where
$$\begin{aligned}
    ||E||_1 &= \Delta x \sum^m_{i=1} |E_i| \\
    ||E||_2 &= \left( \Delta x \sum^m_{i=1} |E_i|^2 \right )^{1/2}
\end{aligned}$$
These are referred to as *grid function norms*.

The $E$ defined above is known as the *global error*.  Our goal now turns to using the local truncation error and some idea of stability to imply the global error goes to zero.

### Local Truncation Error

The *local truncation error* (LTE) can be defined by replacing the approximate solution $U_i$ by the approximate solution $u(x_i)$.  Since the algebraic equations are an approximation to the original BVP, we do not expect that the true solution will exactly satisfy these equations, this resulting difference is the LTE.

For our one-dimensional finite difference approximation from above we have
$$
    \frac{1}{\Delta x^2} (U_{i+1} - 2 U_i + U_{i-1}) = f(x_i),
$$
replacing $U_i$ with $u(x_i)$ in this equation leads to
$$
    \tau_i = \frac{1}{\Delta x^2} (u(x_{i+1}) - 2 u(x_i) + u(x_{i-1})) - f(x_i).
$$

In this form the LTE is not as useful but if we assume $u(x)$ is smooth we can repalce the $u(x_i)$ with their Taylor series counterparts, similar to what we did for finite differences.  The relevant Taylor series are
$$
    u(x_{i \pm 1}) = u(x_i) \pm u'(x_i) \Delta x + \frac{1}{2} u''(x_i) \Delta x^2 \pm \frac{1}{6} u'''(x_i) \Delta x^3 + \frac{1}{24} u^{(4)}(x_i) \Delta x^4 + \mathcal{O}(\Delta x^5)
$$

This leads to an expression for $\tau_i$ of
$$\begin{aligned}
    \tau_i &= \frac{1}{\Delta x^2} \left [u''(x_i) \Delta x^2 + \frac{1}{12} u^{(4)}(x_i) \Delta x^4 + \mathcal{O}(\Delta x^5) \right ] - f(x_i) \\
    &= u''(x_i) + \frac{1}{12} u^{(4)}(x_i) \Delta x^2 + \mathcal{O}(\Delta x^4) - f(x_i) \\
    &= \frac{1}{12} u^{(4)}(x_i) \Delta x^2 + \mathcal{O}(\Delta x^4)
\end{aligned}$$
where we note that the true solution would satisfy $u''(x) = f(x)$.

As long as $ u^{(4)}(x_i) $ remains finite (smooth) we know that $\tau_i \rightarrow 0$ as $\Delta x \rightarrow 0$

We can also write the vector of LTEs as
$$
    \tau = A \hat{U} - F
$$
which implies
$$
    A\hat{U} = F + \tau.
$$

### Global Error

What we really want to bound is the global error $E$.  To relate the global error and LTE we can substitute $E = U - \hat{U}$ into our expression for the LTE to find
$$
    A E = -\tau.
$$
This means that the global error is the solution to the system of equations we difined for the approximation except with $\tau$ as the forcing function rather than $F$!  This also implies that the global error $E$ can be thought of as an approximation to similar BVP as we started with where
$$
    e''(x) = -\tau(x) ~~~ \Omega = [0, 1] \\
    e(0) = 0 ~~~ e(1) = 0.
$$
We can solve this ODE directly by integrating twice since to find to leading order
$$\begin{aligned}
    e(x) &\approx -\frac{1}{12} \Delta x^2 u''(x) + \frac{1}{12} \Delta x^2 (u''(0) + x (u''(1) - u''(0))) \\
    &= \mathcal{O}(\Delta x^2) \\
    &\rightarrow 0 ~~~ \text{as} ~~~ \Delta x \rightarrow 0.
\end{aligned}$$

### Stability

### Consistency

### Convergence